In [1]:
import pandas as pd
import sklearn
import sys
import pickle
import numpy as np

import pandas as pd
import sklearn
import sys
import pickle
import numpy as np
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
import xgboost as xgb

import dash_table as dt

The scikit-learn version is 0.24.1.


### HS CODE 2019

In [2]:
hs_code = pd.read_csv('hs_code_2019_final.csv', dtype='str')
hs_code

,HS_Code,Merged_Description
0,010121,"Live horses, asses, mules and hinnies ;Horses ..."
1,010129,"Live horses, asses, mules and hinnies ;Horses ..."
2,010130,"Live horses, asses, mules and hinnies ;Asses"
3,010190,"Live horses, asses, mules and hinnies ;Other ;..."
4,010221,Live bovine animals ;Cattle ;Purebred breeding...
...,...,...
5516,991921,"Goods of Panama, under the terms of general no..."
5517,991961,"Goods of Panama, under the terms of general no..."
5518,992004,"Goods of Korea, under the terms of general not..."
5519,992038,"Goods of Korea, under the terms of general not..."


### PREDICTION MODEL

In [11]:
count_vector = pickle.load(open('model/xgboost_600/count_vector_200.pickle', "rb" ))
label_enc = pickle.load(open('model/xgboost_600/label_enc_200.pickle', "rb" ))
xgb_model = xgb.Booster(model_file='model/xgboost_600/xgb_model_7')
THRESHOLD = 0.005

In [8]:
def get_sample_prediction(text, num_samples=10) :
    tmp_valid_counts = count_vector.transform([text])
    tmp_predict_da = xgb_model.predict(xgb.DMatrix(tmp_valid_counts.todense()))
    sorted_idx = np.argsort(-tmp_predict_da)
    get_label = lambda x : label_enc.inverse_transform(x)
    classes = get_label(sorted_idx[0])[:num_samples]
    probablities = tmp_predict_da[0][sorted_idx][0][:num_samples]
    
    df_rank = pd.DataFrame([{ 'hs' : c, 'probablity' : p} for c, p in zip(classes, probablities)])
    df_rank = df_rank.merge(hs_code, how='left', left_on='hs', right_on='HS_Code').fillna('No description')
    df_rank.index = np.arange(1, len(df_rank) + 1)
    df_rank.columns = ['hscode', 'probability', 'hscode2', 'desc']
    return df_rank

In [22]:
df_pred = get_sample_prediction(str('bicyclin'), 5)[['hscode', 'probability', 'desc']]
df_pred

,hscode,probability,desc
1,902229,0.001846,Apparatus based on the use of X-rays or of alp...
2,720241,0.001759,Ferroalloys ;Ferrochromium ;Containing by weig...
3,890391,0.001549,Yachts and other vessels for pleasure or sport...
4,853510,0.001528,Electrical apparatus for switching or protecti...
5,293212,0.001521,Heterocyclic compounds with oxygen hetero-atom...


In [23]:
df_pred.query('probability > ' + str(THRESHOLD))

,hscode,probability,desc


### GET BUYERS BY HS CODE

In [16]:
list(df_pred.hs)

['731511',
 '851210',
 '741910',
 '871200',
 '871496',
 '902990',
 '731512',
 '731589',
 '731582',
 '871499']

In [56]:
df_suppl = pd.read_csv('data/consignee_zipcode_df_latlong_V1.csv', dtype='str')

In [57]:
df_suppl.columns

Index(['Consignee Name', 'Consignee Address ', 'Raw Consignee Name',
       'Raw Consignee Addr1', 'Raw Consignee Addr2', 'Raw Consignee Addr3',
       'Raw Consignee Addr4', 'Raw Consignee Addr Others', 'HS Code',
       'HS Code Sure Level', 'Cleaned_HS_Code', 'HS_Code',
       'Merged_Description', 'zipcode', 'postal_code', 'place_name',
       'state_name', 'state_code', 'latitude', 'longitude'],
      dtype='object')

In [58]:
print(len(df_suppl))

1288851


In [59]:
df_suppl.drop_duplicates(subset=['Consignee Name'], inplace=True)
print(len(df_suppl))

333794


In [60]:
df_suppl.to_csv('data/consignee_zipcode_df_latlong_V1_drop_dupes.csv')

In [30]:
df_suppl[['Consignee Name', 'Consignee Address ', 'Cleaned_HS_Code', 'latitude', 'longitude']].dropna(subset=['latitude', 'longitude'])

,Consignee Name,Consignee Address,Cleaned_HS_Code,latitude,longitude
0,SASOL GERMANY GMBH C O LGB CORP,1501 ADMIRAL WILSON BLVD CAMDEN NJ 08109 US,920510,39.9519,-75.0482
1,SASOL GERMANY GMBH C O LGB CORP,1501 ADMIRAL WILSON BLVD CAMDEN NJ 08109 US,281820,39.9519,-75.0482
2,SASOL GERMANY GMBH C O LGB CORP,1501 ADMIRAL WILSON BLVD CAMDEN NJ 08109 US,382499,39.9519,-75.0482
3,MID ATLANTIC INDUSTRIAL TEXTILES IN,7854 BROWNNG ROAD PENNSAUKEN NJ08109 US,481200,39.9519,-75.0482
4,SASOL GERMANY GMBH C O LGB CORP,1501 ADMIRAL WILSON BLVD CAMDEN NJ 08109 US,382499,39.9519,-75.0482
...,...,...,...,...,...
1288843,ISLES BRIDGE LLC,DBA LOCK AND LUBE 21 TECHNOLOGY DRIVE SUITE #2...,846719,36.0366,-84.3486
1288846,PARKER WOLF SELECTIONS,12300 WIRTH DR UNIT A,220410,30.1238,-97.8393
1288847,WINNERS MEATS QUALITY MEATS,8544 STATE ROUTE 705,854091,40.3283,-84.4836
1288848,ALIAGE WALTERS,1006 DORSET DRIVE,980400,41.1285,-75.0132


In [32]:
df_suppl[df_suppl['Cleaned_HS_Code'].isin(list(df_pred.hs))]

,Consignee Name,Consignee Address,Raw Consignee Name,Raw Consignee Addr1,Raw Consignee Addr2,Raw Consignee Addr3,Raw Consignee Addr4,Raw Consignee Addr Others,HS Code,HS Code Sure Level,Cleaned_HS_Code,HS_Code,Merged_Description,zipcode,postal_code,place_name,state_name,state_code,latitude,longitude
700,LEUCOS USA,NC 11 MAYFIELD AVENUE P O BOX 7829 EDISON 0883...,LEUCOS U.S.A. INC.,11 MAYFIELD AVENUE P.O. BOX 7829,EDISON 08837 US,NaN,NaN,NaN,851210,5,851210,851210,Electrical lighting or signaling equipment (ex...,08837,08837,Edison,New Jersey,NJ,40.5325,-74.3375
2052,JOY KIE TECHNOLOGY CO LTD,715 AIRTECH PKWY PLAINFIELD IN 46168 US IND2,"JOY KIE TECHNOLOGY CO.,LIMITED",715 AIRTECH PKWY PLAINFIELD,IN 46168 US (IND2,NaN,NaN,PLAINFIELD IN 46168 US,851210,5,851210,851210,Electrical lighting or signaling equipment (ex...,46168,46168,Plainfield,Indiana,IN,39.6893,-86.3919
2098,RADIO SYSTEMS CORP,3870 S RONALD REAGAN PKWY PLAINFIELPLAINFIELD ...,RADIO SYSTEMS CORP.,3870 S RONALD REAGAN PKWY PLAINFIEL,"PLAINFIELD,46168,US",NaN,NaN,NaN,851210,5,851210,851210,Electrical lighting or signaling equipment (ex...,46168,46168,Plainfield,Indiana,IN,39.6893,-86.3919
2190,JOY KIE TECHNOLOGY CO LTD,715 AIRTECH PKWY PLAINFIELD IN 46168 US,"JOY KIE TECHNOLOGY CO.,LIMITED",715 AIRTECH PKWY,"PLAINFIELD,IN,46168,US",NaN,NaN,NaN,851210,5,851210,851210,Electrical lighting or signaling equipment (ex...,46168,46168,Plainfield,Indiana,IN,39.6893,-86.3919
2432,IWIS,ZIONSVILLE RD INDIANAPOLIS IN 46168 US,IWIS DRIVE SYSTEMS LLC,ZIONSVILLE RD,"INDIANAPOLIS, IN 46168 US",NaN,NaN,NaN,731511,8,731511,731511,"Chain and parts thereof, of iron or steel ;Art...",46168,46168,Plainfield,Indiana,IN,39.6893,-86.3919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288509,VERTILUX LIMITED,7300 N W 35TH TERRACE,VERTILUX LIMITED,7300 N W 35TH TERRACE,NaN,NaN,NaN,MIAMI FL 33120 US,741910,5,741910,741910,Other articles of copper ;Chain and parts thereof,33120,33120,NaN,NaN,NaN,NaN,NaN
1288691,OORAH INC,MUCHEL SCHNUR 964 S GILBOA,OORAH INC,MUCHEL SCHNUR 964 S GILBOA,NaN,NaN,NaN,NEW YORK NY 12076 US,871200,5,871200,871200,Bicycles and other cycles (including delivery ...,12076,12076,Gilboa,New York,NY,42.4108,-74.4003
1288760,BLACK MOUNTAIN CYCLES,11101 STATE ROUTE 1 B,BLACK MOUNTAIN CYCLES,11101,NaN,STATE ROUTE 1 B,NaN,POINT REYES STATION CA 94956 US,902990,5,902990,902990,"Revolution counters, production counters, taxi...",94956,94956,Point Reyes Station,California,CA,38.0691,-122.8069
1288785,IAN CONNOR,PO BOX 287,IAN CONNOR,PO BOX 287,NaN,NaN,NaN,CASANOVA VA 20139 US,871200,5,871200,871200,Bicycles and other cycles (including delivery ...,20139,20139,Casanova,Virginia,VA,38.6543,-77.7025


In [39]:
dt.DataTable(
                style_data={'whiteSpace': 'normal', 'height': 'auto', 'font-size':'16px'},
                id='table',
                columns=[{"name": i, "id": i} for i in df_suppl.head(10).columns],
                data=df_suppl.head(10).to_dict('records'),
                row_selectable='multi'
            )

DataTable(columns=[{'name': 'Consignee Name', 'id': 'Consignee Name'}, {'name': 'Consignee Address ', 'id': 'Consignee Address '}, {'name': 'Raw Consignee Name', 'id': 'Raw Consignee Name'}, {'name': 'Raw Consignee Addr1', 'id': 'Raw Consignee Addr1'}, {'name': 'Raw Consignee Addr2', 'id': 'Raw Consignee Addr2'}, {'name': 'Raw Consignee Addr3', 'id': 'Raw Consignee Addr3'}, {'name': 'Raw Consignee Addr4', 'id': 'Raw Consignee Addr4'}, {'name': 'Raw Consignee Addr Others', 'id': 'Raw Consignee Addr Others'}, {'name': 'HS Code', 'id': 'HS Code'}, {'name': 'HS Code Sure Level', 'id': 'HS Code Sure Level'}, {'name': 'Cleaned_HS_Code', 'id': 'Cleaned_HS_Code'}, {'name': 'HS_Code', 'id': 'HS_Code'}, {'name': 'Merged_Description', 'id': 'Merged_Description'}, {'name': 'zipcode', 'id': 'zipcode'}, {'name': 'postal_code', 'id': 'postal_code'}, {'name': 'place_name', 'id': 'place_name'}, {'name': 'state_name', 'id': 'state_name'}, {'name': 'state_code', 'id': 'state_code'}, {'name': 'latitude', 

### GET SMALL BUSINESSES - SUPPLIERS

In [25]:
hs_code_to_naics = pd.read_csv('hs6_to_naics_mapping.csv', dtype = 'str')
hs_code_to_naics.head()

,Unnamed: 0,hs6,naics
0,0,910211,334518
1,3,160510,311711
2,29,106199,112990
3,39,910212,334518
4,42,910219,334518


In [46]:
hs_code_to_naics[hs_code_to_naics.naics=='332618'][['hs6', 'naics']].to_csv('temp.csv')

In [45]:
list(df_pred.hs)

['731511', '851210', '741910', '871200', '871496']

In [51]:
list(set(hs_code_to_naics[hs_code_to_naics.hs6.isin(list(df_pred.hs))].naics))

['336320', '336321', '332618', '333613', '336991']

In [37]:
import os
os.listdir('data')

['6-digit_2017_Codes.csv',
 'consignee_zipcode_df_latlong_V1.csv',
 'consignee_zipcode_df_latlong_V1_drop_dupes.csv',
 'ppp_loans_naics_lat_long_sample.parq']

In [51]:
df = pd.read_parquet('data/ppp_loans_naics_lat_long_sample.parq') 
df.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'RaceEthnicity', 'UTILITIES_PROCEED', 'PAYROLL_PROCEED',
       'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED',
       'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED', 'BusinessType',
       'OriginatingLenderLocationID', 'OriginatingLender',
       'OriginatingLen

In [55]:
print(df[df.BorrowerName == 'WESTERN STEEL & BOILER CO.'][['LoanNumber', 'DateApproved',
       'BorrowerName', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount']])

        LoanNumber DateApproved                BorrowerName LoanStatusDate  \
838131  9539847200   04/28/2020  WESTERN STEEL & BOILER CO.           None   

         LoanStatus Term SBAGuarantyPercentage InitialApprovalAmount  \
838131  Exemption 4   24                   100               65257.5   

       CurrentApprovalAmount UndisbursedAmount  
838131               65257.5                 0  
